*Import librairies*

In [1]:
import yfinance as yf
import pandas as pd
import csv
from tqdm.notebook import tqdm
import html5lib
import plotly.express as px
from datetime import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

In [16]:
ee = pd.read_csv('covid_USA.csv')

In [18]:
ee.Date = pd.to_datetime(ee.Date)

In [19]:
ee

,Date,Case
0,2020-01-22,1
1,2020-01-23,1
2,2020-01-24,2
3,2020-01-25,2
4,2020-01-26,5
...,...,...
224,2020-09-02,6113510
225,2020-09-03,6150016
226,2020-09-04,6200518
227,2020-09-05,6244970


## ESSAY 1

**Select stock from S&P500**

In [2]:
with open('sp500.csv', newline='') as f:
    reader = csv.reader(f)
    sp500 = [val for sublist in list(reader) for val in sublist]

sp500.remove('GOOGL')
sp500.append('GOOG')
sp500[:15]

['MMM',
 'ABT',
 'ABBV',
 'ABMD',
 'ACN',
 'ATVI',
 'ADBE',
 'AMD',
 'AAP',
 'AES',
 'AFL',
 'A',
 'APD',
 'AKAM',
 'ALK']

**Sectors**

In [3]:
sectors = [
 'Basic Materials',
 'Communication Services',
 'Consumer Cyclical',
 'Consumer Defensive',
 'Energy',
 'Financial Services',
 'Healthcare',
 'Industrials',
 'Real Estate',
 'Technology',
 'Utilities'
]

**Selection**

In [4]:
stocks = [
    'AMZN',
    'GOOG',
    'TSLA'
]

In [5]:
dfs = {}

for stock in stocks:
    dfs[stock] = yf.Ticker(stock).history(period="2y")

In [6]:
dfs['GOOG']

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2018-09-05,1193.80,1199.01,1162.00,1186.48,2061300,0,0
2018-09-06,1186.30,1186.30,1152.00,1171.44,1888500,0,0
2018-09-07,1158.67,1175.26,1157.21,1164.83,1401300,0,0
2018-09-10,1172.19,1174.54,1160.11,1164.64,1115400,0,0
2018-09-11,1161.63,1178.68,1156.24,1177.36,1209300,0,0
...,...,...,...,...,...,...,...
2020-08-31,1647.89,1647.96,1630.31,1634.18,1823400,0,0
2020-09-01,1636.63,1665.73,1632.22,1660.71,1826700,0,0
2020-09-02,1673.78,1733.18,1666.33,1728.28,2511200,0,0


In [7]:
for df in dfs:
    dfs[df]['Returns'] = (dfs[df].Open - dfs[df].Open.shift(1)) / dfs[df].Open.shift(1)
    dfs[df] = dfs[df].iloc[1:, [0,1,2,3,7]]
    dfs[df].reset_index(inplace=True)

In [8]:
dfs['TSLA']

,Date,Open,High,Low,Close,Returns
0,2018-09-06,56.96,58.23,55.78,56.19,-0.000877
1,2018-09-07,52.02,53.67,50.45,52.65,-0.086728
2,2018-09-10,54.65,57.21,54.20,57.10,0.050557
3,2018-09-11,55.89,56.40,54.71,55.89,0.022690
4,2018-09-12,56.29,58.50,55.73,58.11,0.007157
...,...,...,...,...,...,...
499,2020-08-31,444.61,500.14,440.11,498.32,-0.031393
500,2020-09-01,502.14,502.49,470.51,475.05,0.129394
501,2020-09-02,478.99,479.04,405.12,447.37,-0.046103
502,2020-09-03,407.23,431.80,402.00,407.00,-0.149815


**Statistics about stocks**

In [9]:
stats = pd.DataFrame(
    {
        'Std': [dfs[df].Returns.std() for df in dfs],
        'Mean': [dfs[df].Returns.mean() for df in dfs],
        'Min': [dfs[df].Returns.min() for df in dfs],
        'Max': [dfs[df].Returns.max() for df in dfs],
        'Kurtosis': [dfs[df].Returns.kurtosis() for df in dfs],
        'Skewness': [dfs[df].Returns.skew() for df in dfs],
    },
    index=[df for df in dfs]
).T

In [10]:
stats

,AMZN,GOOG,TSLA
Std,0.023637,0.019157,0.048460
Mean,0.001246,0.000796,0.005039
Min,-0.104723,-0.098984,-0.210924
Max,0.104128,0.075776,0.310598
Kurtosis,3.108818,3.653706,5.402544
Skewness,-0.035513,-0.624357,0.631360


In [13]:
stats.to_dict('records')

[{'AMZN': 0.023636652846536894,
  'GOOG': 0.019157349810448983,
  'TSLA': 0.04846044733137317},
 {'AMZN': 0.001246445203032102,
  'GOOG': 0.0007957762347699297,
  'TSLA': 0.005039308337339459},
 {'AMZN': -0.104722891566265,
  'GOOG': -0.09898375610146438,
  'TSLA': -0.2109243697478991},
 {'AMZN': 0.1041280947255112,
  'GOOG': 0.07577648485700729,
  'TSLA': 0.3105981891049428},
 {'AMZN': 3.108817550202903,
  'GOOG': 3.653706489806834,
  'TSLA': 5.402544340020726},
 {'AMZN': -0.0355125198787218,
  'GOOG': -0.6243570115052447,
  'TSLA': 0.6313602015750828}]

**Google Analysis**

In [130]:
fig = px.line(dfs['GOOG'], x='Date', y='Close', title="Google Close Prices")
fig.update_xaxes(rangeslider_visible=True)
fig.update_layout(
    width=1200, 
    height=600,
)

fig.update_layout(
    width=1200, 
    height=600,
    title='GOOG Analysis during the COVID',
    yaxis_title='Stocks',
    shapes = [dict(
        x0='2020-02-15', x1='2020-02-15', y0=0, y1=1, xref='x', yref='paper',
        line_width=2)],
    annotations=[dict(
        x='2020-02-17', y=0.95, xref='x', yref='paper',
        showarrow=False, xanchor='left', text='COVID Begins')],
    yaxis=dict(
    ticksuffix=' $'
    ),
)

fig.write_html("./Basic_Chart/GOOG.html")
fig.show()

**Amazon Analysis**

In [131]:
fig = go.Figure(data=[go.Candlestick(x=dfs['AMZN'].Date,
                open=dfs['AMZN'].Open,
                high=dfs['AMZN'].High,
                low=dfs['AMZN'].Low,
                close=dfs['AMZN'].Close)])

fig.update_layout(
    width=1200, 
    height=800,
    title='AMZN Analysis during the COVID',
    yaxis_title='Stocks',
    shapes = [dict(
        x0='2020-02-15', x1='2020-02-15', y0=0, y1=1, xref='x', yref='paper',
        line_width=2)],
    annotations=[dict(
        x='2020-02-17', y=0.95, xref='x', yref='paper',
        showarrow=False, xanchor='left', text='COVID Begins')],
    yaxis=dict(
    ticksuffix=' $'
    ),
)

fig.write_html("./Basic_Chart/AMZN.html")
fig.show()

**TSLA Analysis**

In [134]:
dfs['TSLA']["Color"] = np.where(dfs['TSLA']['Returns'] < 0, 'red', 'green')

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=dfs['TSLA'].Date, y=dfs['TSLA'].Close, name="Close Prices", yaxis="y"),
    secondary_y=False
)

fig.add_trace(
    go.Bar(x=dfs['TSLA'].Date, y=(dfs['TSLA'].Returns * 100), name="Returns", marker_color=dfs['TSLA'].Color, yaxis="y1"),
    secondary_y=True
)

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
# fig.update_yaxes(title_text="<b>primary</b> yaxis title", secondary_y=False)
# fig.update_yaxes(title_text="<b>secondary</b> yaxis title", secondary_y=True)

fig.update_layout(
    width=1200, 
    height=600,
    title='TSLA Analysis during the COVID',
    yaxis_title='Stocks',
    shapes = [dict(
        x0='2020-02-15', x1='2020-02-15', y0=0, y1=1, xref='x', yref='paper',
        line_width=2)],
    annotations=[dict(
        x='2020-02-17', y=0.95, xref='x', yref='paper',
        showarrow=False, xanchor='left', text='COVID Begins')]
)

fig.update_layout(
    yaxis=dict(
    title="TSLA Close's Prices",
    ticksuffix=' $'

    ),
    yaxis2=dict(
        title="TSLA Returns",
        ticksuffix = '%'
    )
)
fig.write_html("./Basic_Chart/TSLA.html")
fig.show()

**Compare Stocks**

In [133]:
fig = go.Figure()

for df in dfs:
    fig.add_trace(
        go.Scatter(
            x = dfs[df].Date,
            y = dfs[df].Close,
            name = df
        )
    )
    
fig.update_layout(
    updatemenus=[go.layout.Updatemenu(
        active=0,
        buttons=list(
            [dict(label = 'All',
                  method = 'update',
                  args = [{'visible': [True, True, True]},
                          {'title': 'The COVID in USA',
                           'showlegend':True}]),
             dict(label = 'AMZN',
                  method = 'update',
                  args = [{'visible': [True, False, False]},
                          {'title': 'Amazon Stock during the COVID',
                           'showlegend':True}]),             
             dict(label = 'GOOG',
                  method = 'update',
                  args = [{'visible': [False, True, False]},
                          {'title': 'Google Stock during the COVID',
                           'showlegend':True}]),
             dict(label = 'TSLA',
                  method = 'update',
                  args = [{'visible': [False, False, True]},
                          {'title': 'Tesla Stock during the COVID',
                           'showlegend':True}]),
            ])
        )
    ])

fig.update_layout(
    width=1200, 
    height=600,
    title='The COVID in USA (S&P500)',
    yaxis_title='Stocks',
    shapes = [dict(
        x0='2020-02-15', x1='2020-02-15', y0=0, y1=1, xref='x', yref='paper',
        line_width=2)],
    annotations=[dict(
        x='2020-02-17', y=0.95, xref='x', yref='paper',
        showarrow=False, xanchor='left', text='COVID Begins')],
    yaxis=dict(
    ticksuffix=' $'
    ),
)
fig.write_html("./Basic_Chart/Stocks.html")
fig.show()